Часть 1

In [0]:
#!pip install yargy
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from yargy import Parser, rule, or_
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:

game = fact('game', ['name', 'release'])
releases = {'san andreas':'San Andreas', 'sanandreas':'San Andreas',
            'vice city': 'Vice City','vicecity': 'Vice City',
            '5': 'V', 'v': 'V', '4': 'IV', 'iv': 'IV',
            'shift': 'Shift', 'underground': 'Underground', 'most wanted': 'Most Wanted', 'carbon': 'Carbon', 'rivals':'Rivals'}
fifa = rule(morph_pipeline(['fifa', 'фифа', 'фифв', 'фифу']).interpretation(game.name),
            morph_pipeline(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 'STREET']).interpretation(game.release))
NFS = rule(morph_pipeline(['NFS', 'Need for Speed', 'нфс']).interpretation(game.name),
           caseless_pipeline(releases).interpretation(game.release.normalized().custom(releases.get)))
GTA = rule(morph_pipeline(['gta', 'grand theft auto', 'гта']).interpretation(game.name),
           caseless_pipeline(releases).interpretation(game.release.normalized().custom(releases.get)))
GAME = or_(fifa, NFS, GTA).interpretation(game)
parser = Parser(GAME)

In [7]:
matches = []
list(parser.findall('GTA sanandreas'))[0].fact


game(name='GTA',
     release='San Andreas')

In [0]:
for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)
for m in matches:
    print(m.name, m.release)

In [10]:
print(len(matches)) # количество найденных сущностей

1088


In [0]:
# То же самое, но вместе с ".optional" там, где идет информация о части (релизе), что делает ее наличие необязательным
game = fact('game', ['name', 'release'])
releases = {'san andreas':'San Andreas', 'sanandreas':'San Andreas',
            'vice city': 'Vice City','vicecity': 'Vice City',
            '5': 'V', 'v': 'V', '4': 'IV', 'iv': 'IV',
            'shift': 'Shift', 'underground': 'Underground', 'most wanted': 'Most Wanted', 'carbon': 'Carbon', 'rivals':'Rivals'}
fifa = rule(morph_pipeline(['fifa', 'фифа', 'фифв', 'фифу']).interpretation(game.name),
            morph_pipeline(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 'STREET']).interpretation(game.release).optional())
NFS = rule(morph_pipeline(['NFS', 'Need for Speed', 'нфс']).interpretation(game.name),
           caseless_pipeline(releases).interpretation(game.release.normalized().custom(releases.get)).optional())
GTA = rule(morph_pipeline(['gta', 'grand theft auto', 'гта']).interpretation(game.name),
           caseless_pipeline(releases).interpretation(game.release.normalized().custom(releases.get)).optional())
GAME = or_(fifa, NFS, GTA).interpretation(game)
parser = Parser(GAME)

In [18]:
matches = []
list(parser.findall('GTA sanandreas'))[0].fact


game(name='GTA',
     release='San Andreas')

In [0]:
for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)
for m in matches:
    print(m.name, m.release)

In [20]:
print(len(matches))

1449


Можно сделать вывод, что правила для релизов игр в первом случае были прописаны неидеально (Охват 1088 из 1449). В те 1449 штук также могли входить игры без упоминания их подвидов (в объявлениях вроде "продаю приставку с играми NFS, Fifa, GTA"), но думаю, что таких мало, и надо просто смотреть какие еще виды игр и способы их написания существуют. Ещё наверняка были охвачены не все возможные опечатки наподобие "фифв" для "Fifa". 